<tr>
<td style="text-align: center;">
    <div style="display: inline-block;text-align: center">
        <img src="Logo_CentraleSupélec.png" style="width: 30%;">
        <img src="Logo-HeadMind-Partners.png" style="width: 50%;">
    </div>
</td>
</tr>


<h1 style="font-family: Georgia, Arial; font-size: 36px;">Getting the Embedding for the Reference </h1>
<p style="font-size: 24px;">Infonum Project</p>
<p style="font-size: 24px;"></p>
<p style="font-size: 24px;">Members: Francisco García, Olivia Moyal, Ibrahim Ramdane</p>


This is the first step of the pipeline, at the end of this notebook we get the embedding of the reference we choose thus we could use it to compare with the later embeddings of the news to see the similarities with our reference.

We get the reference via the Wikipedia API. This API is free. We define the function with a limit of caracters as the following embedding is limited to 150 tokens (equiv. to 3000 car). For this example, we choose Apple as a Reference.

In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


#### Import Librairies

In [2]:
import requests

#### Context Loader
We start by implementing a function that retreive text document from a wikipedia page using the wikipedia api.

In [3]:
def get_wikipedia_page(page_title: str, max_chars: int = None) -> str:
    endpoint = "https://en.wikipedia.org/w/api.php"
    params = {
        "format": "json",
        "action": "query",
        "prop": "extracts",
        "explaintext": "",
        "titles": page_title
    }

    try:
        response = requests.get(endpoint, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        data = response.json()

        if "query" in data and "pages" in data["query"]:
            pages = data["query"]["pages"]
            page_id = next(iter(pages))
            if "extract" in pages[page_id]:
                full_text = pages[page_id]["extract"]

                if max_chars is not None and len(full_text) > max_chars:
                    return full_text[:max_chars]

                return full_text

    except requests.RequestException as e:
        print(f"Error making API request: {e}")

    return "Unable to fetch Wikipedia page."

#### Context Retreival using Wikipedia

In [18]:
enterprise = "Apple Inc."
max_chars = 2400

In [19]:
reference = get_wikipedia_page(enterprise, max_chars)

In [20]:
print(reference)

Apple Inc. (formerly Apple Computer, Inc.) is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley. It designs, develops, and sells consumer electronics, computer software, and online services. Devices include the iPhone, iPad, Mac, Apple Watch, Vision Pro, and Apple TV; operating systems include iOS, iPadOS, and macOS; and software applications and services include iTunes, iCloud, Apple Music, and Apple TV+.
As of March 2023, Apple was the world's largest company by market capitalization, but it lost this position to Microsoft in January 2024. In 2022, it was the largest technology company by revenue, with US$394.3 billion. As of June 2022, Apple was the fourth-largest personal computer vendor by unit sales, the largest manufacturing company by revenue, and the second-largest manufacturer of mobile phones in the world. It is one of the Big Five American information technology companies, alongside Alphabet (the parent co

#### Get Embedding of the reference

In [7]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


#### Import Librairies

In [8]:
from transformers import AutoTokenizer, AutoModel
import torch

In this case, we choose the model "bert-base" due its best performing results. You can find the results compared to others models in the file "Experiment_models".

In [10]:
model = "bert-base-uncased"
tokenizer_bert = AutoTokenizer.from_pretrained(model)
model_bert = AutoModel.from_pretrained(model)

``get_embedding``   function will convert a text to embedding 

In [16]:
def get_embedding(text):
    # Tokenize and convert to input IDs
    inputs = tokenizer_bert(text, return_tensors="pt")

    # Get the embeddings
    with torch.no_grad():
        outputs = model_bert(**inputs)

    # Mean pooling to get one vector per sequence
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze()


We get the embedding of our reference : Apple

In [21]:
embedding_ref = get_embedding(reference)